In [1]:
import ComputationalHypergraphDiscovery as chd
import pandas as pd
import pickle

import networkx as nx

import torch
torch.manual_seed(0)
import numpy as np


In [2]:
# Parameters
N_datapoints = 1500
sigma = 0.1

# Generate the data
# X -> Y
# X -> Z
# Y,Z-> B
# B -> C
X = torch.randn(N_datapoints)
Y = 4 * torch.sin(2 * X) + sigma * torch.randn(N_datapoints)
Z = 3.5 * X**2 + sigma * torch.randn(N_datapoints)
B = Y**2 - Z**2 / 100 + sigma * torch.randn(N_datapoints)
C = 2 * torch.sin(B) + sigma * torch.randn(N_datapoints)

# Stack the data into a single tensor
data = torch.stack([X, Y, Z, B, C], dim=1)

df = pd.DataFrame(data.numpy(), columns=['X', 'Y', 'Z', 'B', 'C'])
df

,X,Y,Z,B,C
0,-1.125840,-3.067830,4.370651,9.325633,0.016733
1,-1.152360,-2.860869,4.580361,7.944935,2.031671
2,-0.250579,-1.813373,0.153417,3.069329,0.110331
3,-0.433879,-3.111164,0.587666,9.737496,-0.701122
4,0.848710,3.782373,2.491156,14.263285,1.935063
...,...,...,...,...,...
1495,0.403254,3.240962,0.495614,10.383460,-1.694597
1496,0.802490,3.902219,2.322147,15.373322,0.658877
1497,0.262803,2.128708,0.155480,4.623086,-2.091746
1498,0.690109,3.775874,1.733559,14.025502,2.130932


In [3]:
def create_adjacency_matrix(G, weight_key='signal'):
    # nodes 
    nodes = list(G.nodes())
    n = len(nodes)
    
    adj_matrix = np.zeros((n, n))
    for i, node_i in enumerate(nodes):
        for j, node_j in enumerate(nodes):
            if G.has_edge(node_i, node_j):
                adj_matrix[i, j] = G.get_edge_data(node_i, node_j).get(weight_key, 0)
    
    # visualize
    adj_df = pd.DataFrame(adj_matrix, index=nodes, columns=nodes)
    print(adj_df)
    return adj_matrix
    
    adj_matrix_df = create_adjacency_matrix(graph_discovery.G)

In [5]:

try:
    print("loaded data from disk")
    with open("artificial_graph.p", "rb") as file:
        G = pickle.load(file)
except Exception as e:
    print(e)
    graph_discovery=chd.GraphDiscovery.from_dataframe(df)
    G = graph_discovery.G
    graph_discovery.fit()
    
    pickle.dump(graph_discovery.G, open("artificial_graph.p", "wb"))

    # X -> Y
    # X -> Z
    # Y,Z-> B
    # B -> C
    graph_discovery.plot_graph()

loaded data from disk


In [6]:
adj_matrix_df = create_adjacency_matrix(G)

          X         Y         Z         B         C
X  0.000000  0.175792  0.229685  0.011643  0.000000
Y  0.119583  0.000000  0.000000  0.429936  0.000000
Z  0.097555  0.048911  0.000000  0.009265  0.000000
B  0.000000  0.000000  0.000000  0.000000  0.459243
C  0.000000  0.000000  0.000000  0.000000  0.000000


In [7]:
import sys
sys.path.append("../")
from shared_casual_discovery import *

pre_lr = 1e-2
lr = 1e-2
use_lowest_loss = False

mse_losses, model_weights_hist, opt_epoch = find_direction(
                    df, 
                    M=adj_matrix_df.copy(),
                    epochs=2000,
                    pretrain_epochs = 100, pre_lr = pre_lr,
                    d1_weight = 1.0, lr = lr,
                    rng_seed=7,
                    plots=False,
                    use_lowest_loss=use_lowest_loss
                    )

0 0
0


/Users/abvoron/repos/dg2dag/artificial_data/../shared_casual_discovery.py:102: RuntimeWarning: invalid value encountered in scalar divide
  edge_sgn = int(cycle[idx] / edge_idx)


ValueError: cannot convert float NaN to integer